In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import copy
import random
import math

In [215]:
###create test data
inputs = ['10000000','01000000','00100000','00010000','00001000','00000100','00000010','00000001']
outputs = copy.deepcopy(inputs)

In [265]:
sigmoid_derivative(-10)

-110

In [266]:

def setup_arch(example):
    #####setup architecture for N3N neural net
    #####returns parameters
    size_1 = len(example)
    size_2 = 3
    size_3 = size_1
    #####layer 1
    output_1 = example
    weights_1 = []
    bias_1 = []
    for i in range(size_1):
        tmp_weight = []
        tmp_bias = []
        for j in range(size_2):
            tmp_weight.append(random.uniform(-1,1)*0.00005)
            tmp_bias.append(random.uniform(-1,1)*0.00005)
        weights_1.append(tmp_weight)
        bias_1.append(tmp_bias)
    #####layer 2
    output_2 = []
    bias_2 = []
    weights_2 = []
    for i in range(size_2):
        tmp_weight = []
        tmp_bias = []
        for j in range(size_3):
            tmp_weight.append(random.uniform(-1,1)*0.00005)
            tmp_bias.append(random.uniform(-1,1)*0.00005)
        weights_2.append(tmp_weight)
        bias_2.append(tmp_bias)
    #####layer 3 this is output layer
    output_3 = []
    #######format and return
    outputs = [output_1,output_2,output_3]
    sizes = [size_1,size_2,size_3]
    weights = [weights_1,weights_2]
    bias = [bias_1,bias_2]
    ####
    return(outputs,sizes,weights,bias)    
    
def forward_propagate(outputs,sizes,weights,bias):
    #####forward propagate
    ####readin parameters
    output_1 = outputs[0]
    output_2 = []
    output_3 = []
    size_1 = sizes[0]
    size_2 = sizes[1]
    size_3 = sizes[2]
    weights_1 = weights[0]
    weights_2 = weights[1]
    bias_1 = bias[0]
    bias_2 = bias[1]
    ##propagate level 1 to level 2
    for i in range(size_2):
        sum_i = 0.0
        for j in range(size_1):
            #######bias is for neuron
            sum_i += weights_1[j][i]*float(output_1[j])+bias_1[j][i]
        output_2.append(sigmoid(sum_i))
    ##propagate level 2 to level 3
    for i in range(size_3):
        sum_i = 0.0
        for j in range(size_2):
            sum_i += weights_2[j][i]*float(output_2[j])+bias_2[j][i]
        ##transform output using sigmoid function
        output_3.append(sigmoid(sum_i))
    #######modify outputs and return
    outputs[1].append(output_2)
    outputs[2].append(output_3)
    ##
    return

def backpropagate(i,outputs,sizes,weights,bias):
    ####readin parameters
    output_1 = outputs[0]
    output_2 = outputs[1]
    output_3 = outputs[2]
    size_1 = sizes[0]
    size_2 = sizes[1]
    size_3 = sizes[2]
    weights_1 = weights[0]
    weights_2 = weights[1]
    bias_1 = bias[0]
    bias_2 = bias[1]
    ####go backwards first, since last layer is different; layer 3 first
    node_errors_3 = []
    node_errors_2 = []
    node_errors_1 = []
    bias_der_2 = []
    bias_der_1 = []
    weight_der_2 = []
    weight_der_1 = []
    for j in range(size_3):
        error = -(float(output_1[j])-output_3[i][j])*sigmoid_derivative(output_3[i][j])
#         print(error,output_1[j],output_3[i][j],sigmoid_derivative(output_3[i][j]))
        node_errors_3.append(error)
    ###okay calculate using errors from layer 2
    for k in range(size_2):
        error = 0.0
        bias_der_tmp = []
        weight_der_tmp = []
        for j in range(size_3):
            error_tmp = node_errors_3[j]*weights_2[k][j]
            ###update bias der
            bias_der_tmp.append(copy.deepcopy(error_tmp))
            ###update weight der
            weight_der_tmp.append(copy.deepcopy(error * output_2[i][k] ))
            ###update error
            error += copy.deepcopy(error_tmp)
        ##update node errors
        error = error * sigmoid_derivative(output_2[i][k])
        node_errors_2.append(error)
        ##update derivatives
        weight_der_2.append(weight_der_tmp)
        bias_der_2.append(bias_der_tmp)
    ###okay calculate using errors from layer 1
    for k in range(size_1):
        error = 0.0
        bias_der_tmp = []
        weight_der_tmp = []
        for j in range(size_2):
            error_tmp = node_errors_2[j]*weights_1[k][j]
            ###update bias der
            bias_der_tmp.append(copy.deepcopy(error_tmp))
            ###update weight der
            weight_der_tmp.append(copy.deepcopy(error_tmp * float(output_1[k])))
            ###update error
            error += copy.deepcopy(error_tmp)
        ##update node errors
        error = error * sigmoid_derivative(float(output_1[k]))
        node_errors_1.append(error)
        ##update derivatives
        weight_der_1.append(weight_der_tmp)
        bias_der_1.append(bias_der_tmp)
    #######modify outputs and return
    nerrors = [node_errors_1,node_errors_2,node_errors_3]
    bder = [bias_der_1,bias_der_2]
    wder = [weight_der_1,weight_der_2]
    ####
    return nerrors,bder,wder

def sigmoid(x):
    return(1 / (1 + math.exp(-x)))

def sigmoid_derivative(x):
    return(x * (1 - x))

def init_delta_matrices(sizes):
    ####readin parameters
    size_1 = sizes[0]
    size_2 = sizes[1]
    size_3 = sizes[2]
    ####init layers
    layer1 = np.zeros((size_1,size_2))
    layer2 = np.zeros((size_2,size_3))
    ####
    delta_weights = copy.deepcopy([layer1,layer2])
    delta_biases = copy.deepcopy([layer1,layer2])
    ####
    return delta_weights,delta_biases

def train_model(outputs,sizes,weights,bias,weight_decay_coef,step_size):
    ####clear outputs from before
    outputs[1] = copy.deepcopy([])
    outputs[2] = copy.deepcopy([])
    #######propagate forward
    for example in inputs:
        outputs[0] = copy.deepcopy(example)
        forward_propagate(outputs,sizes,weights,bias)
    ######backpropagate 
    nerrors = []
    ######matrix operations use numpy
    delta_weights,delta_biases = init_delta_matrices(sizes)
    ##find errors in weights and bias 
    for i in range(len(inputs)):
        ###backpropagate
        outputs[0] = copy.deepcopy(inputs[i])
        nerror, bder,wder = backpropagate(i,outputs,sizes,weights,bias)
        ###add in weights
#         print('bder',bder)
        np.add(delta_biases[0],bder[0],out=delta_biases[0])
        np.add(delta_biases[1],bder[1],out=delta_biases[1])
        np.add(delta_weights[0],wder[0],out=delta_weights[0])
        np.add(delta_weights[1],wder[1],out=delta_weights[1])
        ###save errors 
        nerrors.append(nerror)
    print(wder[1])
    print(delta_weights[0])
    ########update weights and biases based on sigmoidal gradient descent
    ##layer 1
    tmp_term = np.array(weights[0])-step_size*(np.array(delta_weights[0])/float(len(inputs))+weight_decay_coef*np.array(weights[0]))
    weights[0] = copy.deepcopy(tmp_term).tolist()
    ##layer 2
    tmp_term = np.array(weights[1])-step_size*(np.array(delta_weights[1])/float(len(inputs))+weight_decay_coef*np.array(weights[1]))
    weights[1] = copy.deepcopy(tmp_term).tolist()
    ####update bias
    ##layer 1
    tmp_term = np.array(bias[0])-step_size*(np.array(delta_biases[0])/float(len(inputs)))
    bias[0] = copy.deepcopy(tmp_term).tolist()
    ##layer 2
    tmp_term = np.array(bias[1])-step_size*(np.array(delta_biases[1])/float(len(inputs)))
    bias[1] = copy.deepcopy(tmp_term).tolist()
    ##
    return

In [267]:

weight_decay_coef = 0.01
step_size = 10
iterations = 50
#######setup NN architcture
outputs,sizes,weights,bias = setup_arch(inputs[0])
#######train model for N iterations
for i in range(iterations):
    train_model(outputs,sizes,weights,bias,weight_decay_coef,step_size)
#     print(weights[1])
#     print('outputs:')
#     print(outputs[2])
#     print('next!')

[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
[[-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0], [-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0]]
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 

In [268]:
outputs[2]

[[3.7513872344357737e-06,
  1.2278109650834455e-08,
  1.5946769905717407e-10,
  8.203450299051485e-07,
  1.0542669994804183e-06,
  4.1062003990376605e-11,
  4.414852922611154e-08,
  7.281438137730598e-11],
 [3.770461270978605e-06,
  1.229227127559739e-08,
  1.6048176541745588e-10,
  8.228065018814942e-07,
  1.060628190684996e-06,
  4.125393508686241e-11,
  4.49465636796002e-08,
  7.4060759807555e-11],
 [3.7592776304545887e-06,
  1.2274791227030832e-08,
  1.599458506566552e-10,
  8.212929700990322e-07,
  1.0562826771687657e-06,
  4.116389943239687e-11,
  4.4552885734914765e-08,
  7.3411067870715e-11],
 [3.7539064682368833e-06,
  1.227821371899169e-08,
  1.5965528910079734e-10,
  8.209814423678933e-07,
  1.0550149037609035e-06,
  4.1104591464351634e-11,
  4.4315581912194795e-08,
  7.30804950512483e-11],
 [3.763152774709973e-06,
  1.2282985758155398e-08,
  1.600658720367682e-10,
  8.214370940631186e-07,
  1.0578985786381563e-06,
  4.117407517813551e-11,
  4.4612605891029e-08,
  7.35100535

In [257]:
weights[1]

[[1.5277287605317367e-07,
  9.143064424030597e-08,
  7.20074864001677e-08,
  7.549008635996243e-08,
  1.641298461410166e-07,
  7.38950196275468e-08,
  1.790650463031739e-07,
  7.24787536519595e-08],
 [2.3152187658535006e-07,
  1.5599843070960962e-07,
  2.945764019274902e-08,
  3.5182298277344884e-08,
  7.394762051125811e-08,
  1.0756315842973303e-07,
  -5.229224526644519e-10,
  6.896080777264409e-09],
 [7.96279119235194e-08,
  2.1625293896543826e-07,
  2.5348372350714443e-07,
  2.2968997051291899e-07,
  1.706574329715912e-07,
  1.7418402573741966e-07,
  1.8842411883027218e-07,
  2.399820344096496e-07]]

In [206]:

# for error in all_errors:
nerrors[7]


[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.004449783309853964, 0.004489942897861068, 0.002617795261430389],
  [-0.11959344090266094,
   0.13142711097657853,
   0.13055007624194784,
   0.12873952972204866,
   0.12821973033071815,
   0.13010878331292047,
   0.12969239818059264,
   0.1317610148767752]],
 [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.004229258310120164, 0.004422879726204087, 0.003493376857048991],
  [0.12995384534692417,
   -0.1178234515302919,
   0.1305761875450533,
   0.12873935293427888,
   0.128240930773287,
   0.13016107064078375,
   0.12971329561066305,
   0.13177956097557816]],
 [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.004848785389394437, 0.003624510743430275, 0.0035402834736253737],
  [0.12996953474504333,
   0.13140653716401307,
   -0.1189163204943371,
   0.12873659913846774,
   0.12822519591666232,
   0.13012190962590192,
   0.1296890446051699,
   0.13177614040087735]],
 [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  [0.004243695764262141, 0.00418713

In [162]:
outputs[2]

[[0.5014368197214021,
  0.5027224542533162,
  0.5009718673816362,
  0.5023081774501119,
  0.5024719294133274,
  0.5027910922324559,
  0.503009755198933,
  0.5034785551644291],
 [0.5014368197214021,
  0.5027224542533162,
  0.5009718673816362,
  0.5023081774501119,
  0.5024719294133274,
  0.5027910922324559,
  0.503009755198933,
  0.5034785551644291],
 [0.5014368197214021,
  0.5027224542533162,
  0.5009718673816362,
  0.5023081774501119,
  0.5024719294133274,
  0.5027910922324559,
  0.503009755198933,
  0.5034785551644291],
 [0.5014368197214021,
  0.5027224542533162,
  0.5009718673816362,
  0.5023081774501119,
  0.5024719294133274,
  0.5027910922324559,
  0.503009755198933,
  0.5034785551644291],
 [0.5014368197214021,
  0.5027224542533162,
  0.5009718673816362,
  0.5023081774501119,
  0.5024719294133274,
  0.5027910922324559,
  0.503009755198933,
  0.5034785551644291],
 [0.5014368197214021,
  0.5027224542533162,
  0.5009718673816362,
  0.5023081774501119,
  0.5024719294133274,
  0.502791